### This is my implementation of my homework for Weighted Gene Co-Expression Network Analysis (mostly copied from references)

#### Install Packages ####

Due to the difficulties of disconnecting in Google Colab this code is ran in Kaggle and because Kaggle does't have a content directory I have deleted "/content" everywhere

In [20]:
#system("terminal command", intern=TRUE) #this is how we run terminal commands in R notebooks and see the outputs if there is any output

Installing necssary packages.

In [21]:
#### Install Dependencies for library ####
#### Place your code ####
install.packages("BiocManager") #installing BiocManager because it allows us to install  gplots, WGCNA, DGCA
library("BiocManager") #importing BiocManager 
BiocManager::install("gplots") #installing gplots
BiocManager::install("WGCNA") #installing WGCNA
BiocManager::install("DGCA") #installing DGCA

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: http://cran.rstudio.com/


Bioconductor version 3.12 (BiocManager 1.30.19), R 4.0.5 (2021-03-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'gplots'”
Old packages: 'adegraphics', 'BART', 'basecamb', 'BH', 'BiasedUrn', 'brew',
  'broom', 'caracas', 'cartography', 'cheatsheet', 'chron', 'class', 'clue',
  'collapse', 'colorspace', 'Compind', 'compositions', 'conflicted', 'copula',
  'credentials', 'cubature', 'date', 'devtools', 'doBy', 'dplyr', 'e1071',
  'earth', 'effectsize', 'extrafont', 'fansi', 'ff', 'flexdashboard',
  'flextable', 'fontawesome', 'forcats', 'fpp3', 'FRK', 'fs', 'FSA', 'future',
  'gam', 'gamlss', 'gap', 'gargle', 'geometry', 'ggalluvial', 'ggblanket',
  'ggdist', 'gg

Importing necessary libraries.

In [22]:
library(magrittr)
library(data.table)
library(ggplot2)
library(gplots)
library(WGCNA)
library(DGCA)

#### Load data ####

In [23]:
###################
clin.patient <- read.table("data_clinical_patient.txt", sep = "\t", header = T)
clin.sample <- read.table("data_clinical_sample.txt", sep = "\t", header = T)
gbm.data <- fread("GBM.uncv2.mRNAseq_RSEM_normalized_log2.txt")
###################

In [24]:
dim(gbm.data) #gbm.data dimensions

[1] 18218   172

#### Data preprocessing ####

In [25]:
head(gbm.data) #first 6 rows

gene,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01,TCGA-06-0125-01,TCGA-06-0125-02,TCGA-06-0129-01,TCGA-06-0130-01,⋯,TCGA-41-3915-01,TCGA-41-4097-01,TCGA-41-5651-01,TCGA-76-4925-01,TCGA-76-4926-01,TCGA-76-4927-01,TCGA-76-4928-01,TCGA-76-4929-01,TCGA-76-4931-01,TCGA-76-4932-01
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
?|100133144,1.619742,NA,1.559100,3.9995671,2.475344,3.3110090,1.7552294,4.480310,-0.6449837,⋯,3.040524,3.9321368,0.2856981,3.610700,3.258760,1.2687937,3.346375,3.0833325,2.622930,NA
?|100134869,2.757258,3.9724445,3.801138,3.9027586,2.264506,3.5445348,1.7596672,4.072440,1.5707544,⋯,3.352956,3.7968780,3.3731053,4.117662,3.804147,2.6838536,2.696261,1.7002621,2.992333,3.314726
?|10357,5.773564,4.9724399,5.915141,6.5207962,5.966629,5.7045704,5.3789451,6.252266,5.1321491,⋯,5.152715,4.7880853,6.1310091,6.756695,6.140689,5.4269390,5.407978,5.9594873,5.878205,5.911668
?|10431,9.791685,9.7907955,10.270095,8.8765169,9.093659,8.6144640,8.7116525,9.327291,9.5963955,⋯,9.001455,9.2124876,8.9878174,9.055101,9.199774,9.2781788,9.060000,9.3856470,8.840315,8.964348
?|155060,7.863884,7.3400121,7.077802,8.7060435,8.109671,9.5477068,7.1444788,9.541064,9.1161495,⋯,8.692449,9.0233044,8.2015241,9.147411,8.785122,7.3980998,7.594336,8.3662488,9.730320,7.216806
?|26823,NA,-0.8349564,NA,0.9296395,NA,0.7046064,-0.9428617,2.042819,-0.9553960,⋯,NA,-0.3817613,-0.7143019,-0.222128,NA,-0.3161688,-1.190339,-0.8408875,-1.087427,-1.007232


#### Data preprocessing ####

Some cleaning and selecting only the samples that we need.

In [26]:
exprData <- gbm.data
exprData$gene <- exprData$gene %>% gsub("\\|.*", "", .) #deleting the numbers after "?"
exprData <- exprData[-which(exprData$gene == "?"),] #deleting "?"

n_occur <- data.frame(table(exprData$gene)) #a table of genes
dup_id <- which(exprData$gene %in% n_occur$Var1[n_occur$Freq > 1]) #deleting duplicate gnes
exprData <- as.data.frame(exprData[-dup_id[2],])

exprData[,c(1,which(substr(colnames(exprData), 14, 15) == "01" | substr(colnames(exprData), 14, 15) == "11"))] #only preserving the first column (gene) and columns with sample names having 01, 11 in them

gene,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-02-2486-01,TCGA-06-0125-01,TCGA-06-0129-01,TCGA-06-0130-01,TCGA-06-0132-01,⋯,TCGA-41-3915-01,TCGA-41-4097-01,TCGA-41-5651-01,TCGA-76-4925-01,TCGA-76-4926-01,TCGA-76-4927-01,TCGA-76-4928-01,TCGA-76-4929-01,TCGA-76-4931-01,TCGA-76-4932-01
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A1BG,6.9658639,8.6139876,8.0866787,6.391336,6.7583045,7.6558262,7.0633606,7.25638043,7.0193528,⋯,8.126225,7.5837740,4.7675224,3.8222101,6.6556939,7.3461307,6.523123,7.5898129,5.60992040,5.669324
A2BP1,7.9344546,7.1017247,6.7947927,8.006427,2.0936616,5.7157973,4.1803150,6.49589819,9.9269809,⋯,3.397830,7.6349715,9.2485405,1.0997665,8.3457583,4.0057862,3.333266,3.0661236,7.08246203,2.992805
A2LD1,5.7832576,6.3925579,5.1100792,6.980547,7.5738782,6.8284336,5.1821202,6.26936422,7.3710605,⋯,6.159779,6.8762112,4.6596392,6.7738065,7.4665765,7.1344879,5.979625,4.2123437,6.89638569,6.750095
A2ML1,5.3639210,0.7500919,1.3408471,7.724027,6.6956946,5.3627820,3.4757723,1.62965970,6.8555189,⋯,5.982791,6.8849495,2.2856389,3.3627785,8.3457583,6.6950821,3.763900,8.6709872,7.31341638,4.850784
A2M,15.0537741,15.3878915,14.3621164,12.928987,15.3223305,13.4479789,14.4260279,14.43484944,14.9152719,⋯,16.698109,14.2976144,13.9479174,13.0873568,14.0586465,14.9525308,15.088774,13.9416555,12.85390302,14.408144
A4GALT,5.1804741,8.9299581,6.4423746,7.278339,5.4562279,6.6818512,4.8083490,7.10538265,6.1285367,⋯,6.857259,7.2620187,6.3731070,5.3013755,5.4981775,6.8736791,6.387130,6.7665647,5.23446683,6.015172
A4GNT,0.2978370,-0.8349564,NA,-1.070360,NA,1.0265168,-0.2791087,NA,0.2705887,⋯,2.567764,0.6181447,NA,0.3627785,NA,NA,-1.190339,0.1592417,1.23444001,-1.007232
AAAS,8.8657862,9.2162955,10.1040490,9.985571,9.4549764,9.6960926,10.7247545,9.89487398,8.9323341,⋯,9.598727,9.3237954,9.3980842,10.0718600,9.3633965,9.6682727,9.289482,10.0840471,10.03536520,9.846894
AACSL,-0.7022803,0.1650436,-1.2440015,NA,-0.7135919,1.0265168,-0.2791087,NA,-0.7295309,⋯,NA,1.6181447,-0.7143019,NA,-1.0868138,NA,-1.190339,NA,-0.08742677,NA


Making a table that says if a sample is normal or tumor

In [27]:
############################
exprData <- gbm.data
exprData$gene <- exprData$gene %>% gsub("\\|.*", "", .) #deleting the numbers after "?"
exprData <- exprData[-which(exprData$gene == "?"),] #deleting "?"

n_occur <- data.frame(table(exprData$gene)) #a table of genes
dup_id <- which(exprData$gene %in% n_occur$Var1[n_occur$Freq > 1]) #deleting duplicate gnes
exprData <- as.data.frame(exprData[-dup_id[2],])

exprData <- exprData[,c(1,which(substr(colnames(exprData), 14, 15) == "01" | substr(colnames(exprData), 14, 15) == "11"))] #only preserving the first column (gene) and columns with sample names having 01, 11 in them
rownames(exprData) <- exprData$gene #adding gene names
exprData <- exprData[,-1] %>% na.omit() #removing the gene column because its now redudant (because of previous line, so the final result doesnt have any name for gene column) and also remoing missing (na) values

traitData <- data.frame(sample_id=colnames(exprData), disease_status=substr(colnames(exprData), 14, 15)) #making a dataframe that says what is the disease status for each patient (rows are patient ids and columns are their status)
traitData$Tumor[which(traitData$disease_status == "01")] <- as.numeric("1") #its tumor if disease_status is 01
traitData$Tumor[which(traitData$disease_status == "11")] <- as.numeric("0") #its normal if disease_status is 11
traitData$Normal[which(traitData$disease_status == "01")] <- as.numeric("0") #its tumor if disease_status is 01
traitData$Normal[which(traitData$disease_status == "11")] <- as.numeric("1") #its normal if disease_status is 11
rownames(traitData) <- traitData$sample_id #adding sample_ids to traitData
traitData <- traitData[,-c(1,2)] #reomving the redundant sample_id (the final result doesnt have any column name for sample_ids)
############################

In [28]:
head(traitData) #top 6 rows of traitData

,Tumor,Normal
,<dbl>,<dbl>
TCGA-02-0047-01,1,0
TCGA-02-0055-01,1,0
TCGA-02-2483-01,1,0
TCGA-02-2485-01,1,0
TCGA-02-2486-01,1,0
TCGA-06-0125-01,1,0


#### Construction of Weighted Gene Co-Expression Network (WGCNA) ####


Setting desired settings for WGCNA.

In [29]:
#####################################################################
allowWGCNAThreads(2) #allowing WGCNA to use 2 threads
options(stringsAsFactors = FALSE) #preventing strings to be treated as factor variables because we don't need dummy numbers for non-numeric values

Allowing multi-threading with up to 2 threads.


In [30]:
# exprData transpose
exprData <- as.data.frame(t(exprData)) #transposing exprData (samples becoming rows and genes becoming columns)

In [31]:
head(exprData) #top 6 rows of exprData

,A1BG,A2BP1,A2LD1,A2M,A4GALT,AAAS,AACS,AADAT,AAGAB,AAK1,⋯,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-02-0047-01,6.965864,7.934455,5.783258,15.05377,5.180474,8.865786,8.912540,7.867686,10.006052,10.453281,⋯,8.171274,8.023537,8.659774,6.025751,8.477739,10.114814,10.237409,11.91736,10.448212,9.237409
TCGA-02-0055-01,8.613988,7.101725,6.392558,15.38789,8.929958,9.216296,8.309745,6.640820,10.410639,9.965178,⋯,8.518233,8.869007,7.939874,5.413015,8.130871,9.247236,9.852462,13.48701,9.243237,9.488141
TCGA-02-2483-01,8.086679,6.794793,5.110079,14.36212,6.442375,10.104049,8.945699,8.017969,9.920152,9.440623,⋯,9.106813,9.414139,9.386141,4.279434,8.665767,9.758689,10.239186,12.30686,9.700586,9.462370
TCGA-02-2485-01,6.391336,8.006427,6.980547,12.92899,7.278339,9.985571,8.249283,7.569856,10.358494,9.367363,⋯,8.419459,8.782624,8.787592,5.749789,8.099536,10.394156,10.063395,12.31372,10.154215,9.452192
TCGA-02-2486-01,6.758305,2.093662,7.573878,15.32233,5.456228,9.454976,8.614979,7.762038,10.534825,8.983272,⋯,8.716757,7.381015,6.217041,4.986743,7.630600,9.349699,9.429687,12.92926,9.293331,9.042860
TCGA-06-0125-01,7.655826,5.715797,6.828434,13.44798,6.681851,9.696093,8.815706,8.262992,9.689701,9.671797,⋯,8.550061,8.807426,8.585685,5.854317,8.780050,10.176754,10.271101,13.82542,10.399799,9.511926


In [32]:
# check missing values
#### Place your code ####
which(is.na(exprData)) #seeing if there is any na data in exprData
sum(is.na(exprData)) #seeing the number of na data in exprData

integer(0)

[1] 0

In [33]:
# plot the sample tree
system('mkdir Result') # making a directory called Result
pdf("Result/sampleTree.pdf") #making a pdf called sampleTree.pdf to put the dendogram of data in it
# sampleTree <- #### Place your code / method == average ####
sampleTree <- hclust(dist(exprData), method = "average"); #hierarchical clustering exprData based on the average distance of every gene between each two sample
clust <- cutree(sampleTree, h=215, k=10) #cutting tree in 10 clusters (k overrides h)
options(repr.plot.width=20, repr.plot.height=10)
par(cex = 0.3) #maginification relative the default for text and symbols = 0.3
par(mar = c(0,4,2,0)) #number of lines of margin to be specified on the four sides of the plot
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5, cex.axis = 1.5, cex.main = 2)
abline(h=215, col="red") #adding h=215 line for clustering visually
dev.off()

png 
  2

<img src="Outputs/Images/sampleTree.png" width="400"/>

Clustering

In [34]:
# cut tree
# clust <- #### Place your code / cutHeight = 215, minSize = 10 ####
clust <- cutreeStatic(sampleTree, cutHeight = 215, minSize = 10) #detecting modules in the sampleTree dendogram minimum size of at least 10 (less than 10 is considered noise) 
table(clust) #tabulating clust
keepSamples <- clust == 1 #keeping sampels with clust == 1
exprData <- exprData[keepSamples,] #keeping the samples that we want
traitData <- traitData[-which(rownames(traitData) == "TCGA-06-2569-01"),] #exclusing TCGA-06-2569-01 sample ?

clust
  0   1 
  1 157 

In [35]:
powers = c(c(seq(1, 10, by = 1), seq(12, 20, by = 2)))
powers

[1]  1  2  3  4  5  6  7  8  9 10 12 14 16 18 20

Finding Beta Mean Conncetivity using Soft Thersholding as noise reducer

In [36]:
# choose a soft-threshold power
powers = c(c(seq(1, 10, by = 1), seq(12, 20, by = 2)))
# softThresh <- #### Place your code / indent = 0 ####
softThresh <- pickSoftThreshold(exprData, powerVector = powers, verbose = 5, indent = 0) #making a soft threshold power due to the assumption that raisning correlations to some power will reduce the noise of the correlations in the adjacency matrix
pdf("Result/Beta_MeanConnectivity.pdf") #making pdf
options(repr.plot.width=12, repr.plot.height=20)
par(mfrow = c(1,2))
cex1 = 0.9
plot(softThresh$fitIndices[,1], -sign(softThresh$fitIndices[,3])*softThresh$fitIndices[,2], xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit, signed R^2", type="n", main = paste("Scale independence"));
text(softThresh$fitIndices[,1], -sign(softThresh$fitIndices[,3])*softThresh$fitIndices[,2],labels=powers,cex=cex1,col="blue")
abline(h=0.9, col="red")
plot(softThresh$fitIndices[,1], softThresh$fitIndices[,5], xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",main = paste("Mean connectivity"))
text(softThresh$fitIndices[,1], softThresh$fitIndices[,5], labels=powers, cex=cex1,col="blue")
dev.off()

pickSoftThreshold: will use block size 3137.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3137 of 14259
   ..working on genes 3138 through 6274 of 14259
   ..working on genes 6275 through 9411 of 14259
   ..working on genes 9412 through 12548 of 14259
   ..working on genes 12549 through 14259 of 14259
   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1    0.101  1.080          0.969 2600.000  2.54e+03 4480.0
2      2    0.219 -0.871          0.949  750.000  6.83e+02 1970.0
3      3    0.658 -1.640          0.967  276.000  2.26e+02 1060.0
4      4    0.760 -2.040          0.962  119.000  8.55e+01  647.0
5      5    0.789 -2.200          0.967   57.800  3.56e+01  423.0
6      6    0.803 -2.250          0.975   30.700  1.59e+01  291.0
7      7    0.814 -2.260          0.982   17.500  7.49e+00  207.0
8      8    0.807 -2.260          0.984   10.500  3.74e+00  151.0
9      9    0.814 -2.200          0.985    6.660  1

png 
  2

<img src="Outputs/Images/Beta_MeanConnectivity.png" width="400"/>

Doing WGCNA and using Eigenvectors for hub gene identification for tumor

In [37]:
# construction of weighted gene co-expression network
# net <- #### Place your code / saveTOMFileBase = "/content/Result/TOM" / power = 14 / minModuleSize = 30 / mergeCutHeight = 0.25####
net <- blockwiseModules(exprData, power = 14, TOMType = "unsigned", minModuleSize = 30, reassignThreshold = 0, mergeCutHeight = 0.25, numericLabels = TRUE, pamRespectsDendro = FALSE, saveTOMs = TRUE, saveTOMFileBase = "Result/TOM", verbose = 3) #automatic network construction and module detection on large expression datasets in a block-wise manner
mergedColors <- labels2colors(net$colors)

pdf("Result/clusterDendrogram.pdf")
options(repr.plot.width=14, repr.plot.height=20) 
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]], "Module\nColors", dendroLabels = FALSE, hang = 0.03, addGuide = TRUE, guideHang = 0.05) #plotting dendogram
dev.off()
 
moduleLabels = net$colors
# moduleColors = #### Place your code ####
moduleColors = labels2colors(net$colors)
# MEs = #### Place your code ####
MEs = net$MEs
geneTree = net$dendrograms[[1]]
nGenes = ncol(exprData)
nSamples = nrow(exprData)
# MEs0 = #### Place your code ####
MEs0 = moduleEigengenes(exprData, moduleColors)$eigengenes #calculating module eigengenes (1st principal component) of modules for hub genes
MEs = orderMEs(MEs0) #reordering given eigenvectors such that similar ones (as measured by correlation) are next to each other
moduleTraitCor = cor(MEs, traitData, use = "p") #correlation of MEs and traitData
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples) #calculatign Student asymptotic p-value for given correlations

pdf("Result/module_trait_relationship.pdf")
options(repr.plot.width=14, repr.plot.height=14)
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",signif(moduleTraitPvalue, 1), ")", sep = "")
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3))
labeledHeatmap(Matrix = moduleTraitCor, xLabels = names(traitData), yLabels = names(MEs), ySymbols = names(MEs), colorLabels = FALSE,
               colors = blueWhiteRed(50),
               textMatrix = textMatrix, setStdMargins = FALSE, cex.text = 0.5, zlim = c(-1,1), main = paste("Module-trait relationships"))
dev.off()

Tumor <- as.data.frame(traitData$Tumor)
Normal <- as.data.frame(traitData$Normal)
names(Tumor) <- "Tumor"
names(Normal) <- "Normal"

# *** Modules with |ME| > 0.5 were selected ***

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
 ....pre-clustering genes to determine blocks..
   Projective K-means:
   ..k-means clustering..
   ..merging smaller clusters...
Block sizes:
gBlocks
   1    2    3 
4996 4974 4289 
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file Result/TOM-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
 ..Working on block 2 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 2 into file R

png 
  2

png 
  2

<img src="Outputs/Images/clusterDendrogram.png" width="400"/>
<img src="Outputs/Images/module_trait_relationship.png" width="400"/>

Analyzing for the brown module accuried from previous method 

In [38]:
# Brown Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
# MMPvalue <- as.data.frame(#### Place your code ####)
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
# geneTraitSignificance <- as.data.frame(#### Place your code ####)
geneTraitSignificance <- as.data.frame(cor(exprData, Normal, use = "p")) #in original code it was weight
# GSPvalue <- as.data.frame(#### Place your code ####)
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))  #+
names(geneTraitSignificance) <- paste("GS.", names(Normal), sep="")
names(GSPvalue) <- paste("p.GS.", names(Normal), sep="")

In [39]:
# identifying genes with high GS and MM
module <- "brown"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("Result/scatterBrown.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Normal",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

<img src="Outputs/Images/scatterBrown.png" width="400"/>

Analyzing for the yellow module accuried from previous method 

In [40]:
# Yellow Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
# MMPvalue <- as.data.frame(#### Place your code ####)
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
# geneTraitSignificance <- as.data.frame(#### Place your code ####)
geneTraitSignificance <- as.data.frame(cor(exprData, Tumor, use = "p"))
# GSPvalue <- as.data.frame(#### Place your code ####)
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) <- paste("GS.", names(Tumor), sep="")
names(GSPvalue) <- paste("p.GS.", names(Tumor), sep="")

In [41]:
# identifying genes with high GS and MM
module <- "yellow"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("Result/scatterYellow.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Tumor",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

<img src="Outputs/Images/scatterYellow.png" width="400"/>

Analyzing for the grey module accuried from previous method 

In [42]:
# Grey Module
modNames <- substring(names(MEs), 3)
geneModuleMembership <- as.data.frame(cor(exprData, MEs, use = "p"))
# MMPvalue <- as.data.frame(#### Place your code ####)
MMPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples))
names(geneModuleMembership) <- paste("MM", modNames, sep="")
names(MMPvalue) <- paste("p.MM", modNames, sep="")
# geneTraitSignificance <- as.data.frame(#### Place your code ####)
geneTraitSignificance <- as.data.frame(cor(exprData, Normal, use = "p"))
# GSPvalue <- as.data.frame(#### Place your code ####)
GSPvalue <- as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples))
names(geneTraitSignificance) <- paste("GS.", names(Normal), sep="")
names(GSPvalue) <- paste("p.GS.", names(Normal), sep="")

In [43]:
# identifying genes with high GS and MM
module <- "grey"
column <- match(module, modNames)
moduleGenes <- moduleColors == module
pdf("Result/scatterGrey.pdf")
par(mfrow = c(1,1))
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]), abs(geneTraitSignificance[moduleGenes, 1]),
                   xlab = paste("Module Membership in", module, "module"),
                   ylab = "Gene significance for Normal",
                   main = paste("Module membership vs. gene significance\n"),
                   cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

# *** Module with r > 0.5 was selected ***
yellowModuleGenes <- colnames(exprData)[moduleColors == "yellow"]
#####################################################################

png 
  2

<img src="Outputs/Images/scatterGrey.png" width="400"/>

#### Differential Gene Correlation Analysis (DGCA) ####


loading the data again because we need untouched data again

In [44]:
###################
clin.patient <- read.table("data_clinical_patient.txt", sep = "\t", header = T)
clin.sample <- read.table("data_clinical_sample.txt", sep = "\t", header = T)
gbm.data <- fread("GBM.uncv2.mRNAseq_RSEM_normalized_log2.txt")
###################

Doing some preprocessing  ony one tumor ones and this time using Mesenchymal and NonMesenchymal for our analysis on tumor samples then doing the differential analysis 

In [45]:
#######################################################
#Preprocessings like the previous section except this time we only keep tumor data
gbm.data$gene <- gbm.data$gene %>% gsub("\\|.*", "", .)
gbm.data <- as.data.frame(gbm.data[-which(gbm.data$gene == "?"),])
gbm.data <- gbm.data[,c(1,which(substr(colnames(gbm.data), 14, 15) == "01"))]

n_occur <- data.frame(table(gbm.data$gene))
dup_id <- which(gbm.data$gene %in% n_occur$Var1[n_occur$Freq > 1])
gbm.data <- gbm.data[-dup_id[2],] # removing duplicate symbol

rownames(gbm.data) <- gbm.data$gene
gbm.data <- gbm.data[,-1]

clin.sample <- clin.sample[,c(2,3)] %>% na.omit()

Mesenchymal <- clin.sample$SAMPLE_ID[which(clin.sample$EXPRESSION_SUBTYPE == "Mesenchymal")] #Mesenchymal data in clin.sample (some specific stem cells in the brain)
NonMesenchymal <- clin.sample$SAMPLE_ID[which(clin.sample$EXPRESSION_SUBTYPE != "Mesenchymal")] #non Mesenchymal data in clin.sample

#creating a design matrix with rows being samples and columns being features (Mesenchymal, NonMesenchymal)
design <- data.table(subtype=rep("", length(colnames(gbm.data)))) #empty columns
design <- as.data.frame(design) #making dataframe from it
design[which(colnames(gbm.data) %in% Mesenchymal), "subtype"] <- "Mesenchymal"
design[which(colnames(gbm.data) %in% NonMesenchymal), "subtype"] <- "NonMesenchymal"
subtype <- design$subtype
design_matrix <- model.matrix(~ subtype + 0) #design matrix creation
design_matrix <- design_matrix[,-1] #removing the extra column
colnames(design_matrix) <- c("Mesenchymal", "NonMesenchymal") #renaming

gbm.data <- gbm.data[which(rownames(gbm.data) %in% yellowModuleGenes),] #only keeping tumor data ?

# ddcor_res = #### Place your code /compare between "Mesenchymal", "NonMesenchymal" / heatmapPlot = F ####
ddcor_res = ddcorAll(inputMat = gbm.data, design = design_matrix, compare = colnames(design_matrix), heatmapPlot = FALSE) #this is the automatic pipleline for differential correlation for comparing pairwise correlations across conditions in the Differential Gene Correlation for colnames which Mesenchymal/NonMesenchymal. This function gives as all the correlations and p values for every combination of two genes. it also uses z transformed correlation
View(ddcor_res)

#now this time were writing the ddcorAll pipeline step by step by ourselves
cor_res <- getCors(inputMat = gbm.data, design = design_matrix) #claculating the correlations between every two genes (all the combinations)
# pairDCor_res <- pairwiseDCor(#### Place your code ####)
pairDCor_res <- pairwiseDCor(cor_res, compare = colnames(design_matrix)) #pairting correlations for cor_res and colnames (Mesenchymal, NonMesenchymal)
dc_TopPairs = dcTopPairs(pairDCor_res, nPairs = 100128, classify = TRUE, adjust = "fdr", plotFdr = F) #all gene pairs (or just the top n pairs), sorted by their differential correlation p-value and adjusintg pvalues with fdr method
dc_TopPairs <- dc_TopPairs[which(dc_TopPairs$pValDiff_adj < 0.05),] #filtering the ones woth p value >= 0.05
View(dc_TopPairs)
write.csv(dc_TopPairs, "Result/differentially_co-expression_gene_network.csv") #drawing using cytoscape

pdf("Result/heatmapPlot.pdf")
options(repr.plot.width = 18, repr.plot.height =20, repr.plot.res = 150)
# gbm.data_top = #### Place your code / use filterGenes with filterTypes = c("central", "dispersion") ####
gbm.data_top = filterGenes(gbm.data, filterTypes = c("central", "dispersion"), filterCentralPercentile = 0.75, filterDispersionPercentile = 0.75) #filtering genes with CentralPercentile = 0.75 and filterDispersionPercentile = 0.75
# ddcor_res = #### Place your code / As above / nPairs = "all" ####
ddcor_res = ddcorAll(inputMat = gbm.data_top, design = design_matrix,  compare = colnames(design_matrix), heatmapPlot = TRUE,  nPairs = "all", adjust = "BH") #doing ddcorAll for only filtered genes
dev.off()

pdf("Result/plotCors.pdf") #plotting the correlation for each two genes in Mesenchymal and NonMesenchymal samples in two separate plots
options(repr.plot.width = 18, repr.plot.height = 12, repr.plot.res = 150)
plotCors(inputMat = gbm.data, design = design_matrix, compare = c("Mesenchymal", "NonMesenchymal"), geneA = "KIF20A", geneB = "TACC3")
dev.off()
#######################################################

Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.



,Gene1,Gene2,Mesenchymal_cor,Mesenchymal_pVal,NonMesenchymal_cor,NonMesenchymal_pVal,zScoreDiff,pValDiff,empPVals,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF20A,TACC3,0.92555679,0.000000e+00,0.59512326,4.213563e-11,-5.233454,1.663710e-07,7.390540e-05,0.6253636,+/+
2,CCNA2,TACC3,0.89753257,0.000000e+00,0.49081623,1.630576e-07,-5.129677,2.902401e-07,8.788750e-05,0.6253636,+/+
3,LBR,LOC730101,-0.06237795,6.736193e-01,0.68997816,8.881784e-16,5.063631,4.113460e-07,9.387983e-05,0.6253636,0/+
4,FAM83D,TACC3,0.88072067,0.000000e+00,0.45191647,1.861864e-06,-4.960687,7.024443e-07,1.038671e-04,0.6253636,+/+
5,ITPR1,POLR3D,0.45719107,1.087634e-03,-0.36469899,1.642034e-04,-4.872717,1.100738e-06,1.148530e-04,0.6253636,+/-
6,PLK1,TACC3,0.87915142,2.220446e-16,0.46095445,1.085745e-06,-4.858515,1.182695e-06,1.188479e-04,0.6253636,+/+
7,SPC25,TACC3,0.85477035,1.065814e-14,0.38112590,7.750565e-05,-4.851438,1.225694e-06,1.218440e-04,0.6253636,+/+
8,EFR3A,REEP5,-0.02435999,8.694584e-01,0.68815405,1.332268e-15,4.832426,1.348791e-06,1.258389e-04,0.6253636,0/+
9,NCAPG,TACC3,0.90139514,0.000000e+00,0.54900512,2.307432e-09,-4.798217,1.600845e-06,1.378236e-04,0.6253636,+/+


Classifying the differential correlation calls.



,Gene1,Gene2,corA,corA_pVal,corB,corB_pVal,zScoreDiff,pValDiff,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF20A,TACC3,0.92555679,0.000000e+00,0.59512326,4.213563e-11,-5.233454,1.663710e-07,0.01372908,+/+
2,CCNA2,TACC3,0.89753257,0.000000e+00,0.49081623,1.630576e-07,-5.129677,2.902401e-07,0.01372908,+/+
3,LBR,LOC730101,-0.06237795,6.736193e-01,0.68997816,8.881784e-16,5.063631,4.113460e-07,0.01372908,0/+
4,FAM83D,TACC3,0.88072067,0.000000e+00,0.45191647,1.861864e-06,-4.960687,7.024443e-07,0.01388049,+/+
5,ITPR1,POLR3D,0.45719107,1.087634e-03,-0.36469899,1.642034e-04,-4.872717,1.100738e-06,0.01388049,+/-
6,PLK1,TACC3,0.87915142,2.220446e-16,0.46095445,1.085745e-06,-4.858515,1.182695e-06,0.01388049,+/+
7,SPC25,TACC3,0.85477035,1.065814e-14,0.38112590,7.750565e-05,-4.851438,1.225694e-06,0.01388049,+/+
8,EFR3A,REEP5,-0.02435999,8.694584e-01,0.68815405,1.332268e-15,4.832426,1.348791e-06,0.01388049,0/+
9,NCAPG,TACC3,0.90139514,0.000000e+00,0.54900512,2.307432e-09,-4.798217,1.600845e-06,0.01388049,+/+


Warning message in ddcorAll(inputMat = gbm.data_top, design = design_matrix, compare = colnames(design_matrix), :
“If you are not choosing permutation for p-value adjustment or calculating the differential correlation average, then you may be wasting time by generating permutation samples. Consider setting nPerms to 0.”
Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Classifying the differential correlation calls.

Plotting heatmap using heatmap.2...



png 
  2

png 
  2

<img src="Outputs/Images/heatmapPlot.png" width="400"/>
<img src="Outputs/Images/plotCors.png" width="400"/>

#### Differential Gene Correlation Analysis (DGCA) based on AGE ####


In this section We're exactly doing what we did in the previous Defferenail Gene Correlation Analysis except this time we're doing it based on age and not Mesenchymality.

In [46]:
clin.patient <- clin.patient[,c(1,2)] %>% na.omit() #deleting na ones
clin.patient

,PATIENT_ID,AGE
,<chr>,<dbl>
1,TCGA-02-0001,44.3
2,TCGA-02-0003,50.2
3,TCGA-02-0004,59.2
4,TCGA-02-0006,56.2
5,TCGA-02-0007,40.5
6,TCGA-02-0009,61.5
7,TCGA-02-0010,20.4
8,TCGA-02-0011,19.0
9,TCGA-02-0014,25.6


In [47]:
####################################################################

clin.patient <- clin.patient[,c(1,2)] %>% na.omit() #deleting na ones now we have which is sample and its age
clin.sample["SAMPLE_ID"] <- lapply(clin.sample["SAMPLE_ID"], function(x) substr(x, 1, 12)) #having clin.sample which is sample and its subtipe
sample.age <- data.table(SAMPLE_ID=clin.sample[which(clin.sample$SAMPLE_ID %in% clin.patient$PATIENT_ID), 1], AGE=clin.patient$AGE)

young <- sample.age$SAMPLE_ID[which(sample.age$AGE <= median(sample.age$AGE))]
old <- sample.age$SAMPLE_ID[which(sample.age$AGE > median(sample.age$AGE))]

Warning message in as.data.table.list(x, keep.rownames = keep.rownames, check.names = check.names, :
“Item 1 has 523 rows but longest item has 542; recycled with remainder.”


In [48]:
Temp = colnames(gbm.data)
Temp = lapply(Temp, FUN=function(x) substr(x, 1, 12))

In [49]:
design <- data.table(age=rep("", length(colnames(gbm.data))))
design <- as.data.frame(design)
design[which(Temp %in% young), "age"] <- "young"
design[which(Temp %in% old), "age"] <- "old"

In [50]:
age <- design$age

In [51]:
age 

[1] "old"   "old"   "young" "young" "old"   "old"   "young" "young" "young"
 [10] "young" "old"   "young" "old"   "old"   "young" "young" "young" "old"  
 [19] "old"   "old"   "old"   "old"   "old"   "old"   "old"   "old"   "young"
 [28] "old"   "young" "young" "old"   "old"   "old"   "young" "young" "old"  
 [37] "young" "old"   "young" "old"   "old"   "old"   "young" "old"   "old"  
 [46] "young" "young" "old"   "young" "young" "young" "old"   "young" "old"  
 [55] "old"   "old"   "old"   "young" "young" "old"   "young" "young" "young"
 [64] "old"   "young" "young" "young" "young" "old"   "young" "old"   "young"
 [73] "young" "old"   "old"   "young" "old"   "young" "old"   "young" "young"
 [82] "old"   "old"   "old"   ""      "young" "young" "young" "old"   "old"  
 [91] "old"   "young" "young" "old"   ""      "young" "old"   "old"   "old"  
[100] "old"   "old"   "young" "young" "old"   "young" "young" "young" "young"
[109] "old"   "old"   "old"   "old"   "old"   "young" "young" "old"   "young"
[118] ""      "old"   ""      "old"   "old"   "old"   "young" "old"   "young"
[127] "old"   "young" "young" "old"   "young" "old"   "old"   "young" "old"  
[136] "young" "old"   "old"   "young" "old"   "old"   "old"   "young" "old"  
[145] "young" "old"   "old"   "old"   "young" "young" "old"   "young" "young"

In [52]:
design_matrix <- model.matrix(~ age + 0)

In [53]:
design_matrix <- design_matrix[,-1]
# colnames(design_matrix) <- #### Place your code ####
colnames(design_matrix) <- c("old", "young")

Doing ddcorAll like the MesenChymal/NonMesenChymal case but this time for young and old and also specifically for two genes CDCA8 and FAM83D

In [56]:
# ddcor_res = #### Place your code / compare = c("old", "young") #### 
ddcor_res = ddcorAll(inputMat = gbm.data, design = design_matrix, compare = colnames(design_matrix), heatmapPlot = FALSE)
View(ddcor_res)

Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Calculating empirical p-values using the permutation sample statistics.

Sorting the combination of the actual and permuted test statistics.

Finding the proportion of each actual test statistic greater than the permuted test statistics.

Calculating qvalues from the empirical p-values.

Classifying the differential correlation calls.



,Gene1,Gene2,old_cor,old_pVal,young_cor,young_pVal,zScoreDiff,pValDiff,empPVals,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF2C,TACC3,0.90387501,0.000000e+00,0.52045698,4.562227e-06,-5.460848,4.738651e-08,9.987216e-06,0.999963,+/+
2,BUB1,TACC3,0.85171113,0.000000e+00,0.44307424,1.375201e-04,-4.687473,2.765997e-06,7.790029e-05,0.999963,+/+
3,TACC3,TPX2,0.87695353,0.000000e+00,0.54690850,1.161354e-06,-4.461891,8.123962e-06,1.468121e-04,0.999963,+/+
4,CDCA8,TACC3,0.85271120,0.000000e+00,0.47713687,3.396034e-05,-4.451593,8.523548e-06,1.478108e-04,0.999963,+/+
5,RTN4,TBRG1,0.56576351,4.535713e-08,-0.08198759,5.030317e-01,-4.312737,1.612462e-05,1.967482e-04,0.999963,+/0
6,HJURP,TACC3,0.85364153,0.000000e+00,0.49834078,1.315380e-05,-4.306096,1.661607e-05,1.987456e-04,0.999963,+/+
7,CENPE,TACC3,0.83979715,0.000000e+00,0.48019652,2.973055e-05,-4.156568,3.230644e-05,2.916267e-04,0.999963,+/+
8,AURKB,TACC3,0.78463810,0.000000e+00,0.34784261,3.404268e-03,-4.139238,3.484615e-05,3.125999e-04,0.999963,+/+
9,CKAP2L,TACC3,0.82624471,0.000000e+00,0.44806738,1.130553e-04,-4.136655,3.524048e-05,3.145973e-04,0.999963,+/+


In [55]:
cor_res <- getCors(inputMat = gbm.data, design = design_matrix)
pairDCor_res <- pairwiseDCor(cor_res, compare = c("old", "young"))
dc_TopPairs = dcTopPairs(pairDCor_res, nPairs = 100128, classify = TRUE, adjust = "fdr", plotFdr = F)
dc_TopPairs <- dc_TopPairs[which(dc_TopPairs$pValDiff_adj < 0.05),]
View(dc_TopPairs)
write.csv(dc_TopPairs, "Result/AGE_differentially_co-expression_gene_network.csv") #drawing using cytoscape

pdf("Result/AGE_heatmapPlot.pdf")
options(repr.plot.width = 18, repr.plot.height =20, repr.plot.res = 150)
# gbm.data_top =  #### Place your code / filterGenes with filterTypes = c("central", "dispersion") ####
gbm.data_top = filterGenes(gbm.data, filterTypes = c("central", "dispersion"), filterCentralPercentile = 0.75, filterDispersionPercentile = 0.75)
# ddcor_res = #### Place your code /nPairs = "all" #### 
ddcor_res = ddcorAll(inputMat = gbm.data_top, design = design_matrix, compare = colnames(design_matrix), heatmapPlot = TRUE, nPairs = "all", adjust = "BH")
dev.off()

pdf("Result/AGE_plotCors.pdf")
options(repr.plot.width = 18, repr.plot.height = 12, repr.plot.res = 150)
plotCors(inputMat = gbm.data, design = design_matrix, compare = c("old", "young"), geneA = "CDCA8", geneB = "FAM83D") #now comparing for ond and young between CDCA8 and FAM83D genes
dev.off()
####################################################################

Classifying the differential correlation calls.



,Gene1,Gene2,corA,corA_pVal,corB,corB_pVal,zScoreDiff,pValDiff,pValDiff_adj,Classes
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,KIF2C,TACC3,0.903875,0,0.520457,4.562227e-06,-5.460848,4.738651e-08,0.004744716,+/+


Warning message in ddcorAll(inputMat = gbm.data_top, design = design_matrix, compare = colnames(design_matrix), :
“If you are not choosing permutation for p-value adjustment or calculating the differential correlation average, then you may be wasting time by generating permutation samples. Consider setting nPerms to 0.”
Calculating permutation number 1.

Calculating permutation number 2.

Calculating permutation number 3.

Calculating permutation number 4.

Calculating permutation number 5.

Calculating permutation number 6.

Calculating permutation number 7.

Calculating permutation number 8.

Calculating permutation number 9.

Calculating permutation number 10.

Classifying the differential correlation calls.

Plotting heatmap using heatmap.2...



png 
  2

png 
  2

<img src="Outputs/Images/AGE_heatmapPlot.png" width="400"/>
<img src="Outputs/Images/AGE_plotCors.png" width="400"/>

Every Code in in and outputs is in HW4_Cgenomics.ipynb and all the outputs are in Outputs folder and references are in References.txt

In [ ]:
#References
#https://simplystatistics.org/posts/2015-07-24-stringsasfactors-an-unauthorized-biography/
#https://stat.ethz.ch/R-manual/R-patched/library/stats/html/cutree.html
#https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/par
#https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/abline
#https://www.rdocumentation.org/packages/WGCNA/versions/1.13/topics/cutreeStatic
#https://github.com/andymckenzie/DGCA/blob/master/tests/testthat/test-dCorAvg.R
#https://www.rdocumentation.org/packages/WGCNA/versions/1.68/topics/corPvalueStudent
#https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/paste
#https://www.rdocumentation.org/packages/DGCA/versions/1.0.2/topics/ddcorAll
#https://www.rdocumentation.org/packages/DGCA/versions/1.0.2/topics/dcTopPairs
#https://stackoverflow.com/questions/41598916/resize-the-image-in-jupyter-notebook-using-markdown